In [0]:
import os
import numpy as np
import pandas as pd

import os
import cv2
from pathlib import Path
import random

from skimage.io import imsave, imread
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization,UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

from tensorflow.python.framework import ops
ops.reset_default_graph()


In [6]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [7]:
#This csv loads the info that will become the masks which is in run-length encoding
train_df = pd.read_csv('/content/drive/My Drive/Steel/train.csv')
train_df.shape


(50272, 2)

In [8]:
im_idx_list = []
im_idx_list_onlydf_1 = []
im_idx_list_onlydf_2 = []
im_idx_list_onlydf_3 = []
im_idx_list_onlydf_4 = []
im_idx_list_onlydf_5 = []



for col in range(0, len(train_df), 4):
    img_names = [str(i).split("_")[0] for i in train_df.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError
        
    
    
    labels = train_df.iloc[col:col+4, 1]
    if labels.isna().all():
        im_idx_list.append(col)
        
        
    elif (labels.isna() == [False, True, True, True]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf_1.append(col)
        
    elif (labels.isna() == [True, False, True, True]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf_2.append(col)
        
    elif (labels.isna() == [True, True, False, True]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf_3.append(col)
        
    elif (labels.isna() == [True, True, True, False]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf_4.append(col)
        
    else:
        im_idx_list.append(col)
        im_idx_list_onlydf_5.append(col)
        
r=88    
im_idx_list_onlydf_1, im_idx_list_onlydf_val_1 = train_test_split(im_idx_list_onlydf_1, test_size=.2, random_state=r)
im_idx_list_onlydf_2, im_idx_list_onlydf_val_2 = train_test_split(im_idx_list_onlydf_2, test_size=.2, random_state=r)
im_idx_list_onlydf_3, im_idx_list_onlydf_val_3 = train_test_split(im_idx_list_onlydf_3, test_size=.2, random_state=r)
im_idx_list_onlydf_4, im_idx_list_onlydf_val_4 = train_test_split(im_idx_list_onlydf_4, test_size=.2, random_state=r)
im_idx_list_onlydf_5, im_idx_list_onlydf_val_5 = train_test_split(im_idx_list_onlydf_5, test_size=.2, random_state=r)

im_idx_list_onlydf = im_idx_list_onlydf_1 + im_idx_list_onlydf_2 + im_idx_list_onlydf_3 + im_idx_list_onlydf_4 + im_idx_list_onlydf_5
im_idx_list_onlydf_val = im_idx_list_onlydf_val_1 + im_idx_list_onlydf_val_2 + im_idx_list_onlydf_val_3 + im_idx_list_onlydf_val_4 + im_idx_list_onlydf_val_5

print(len(im_idx_list_onlydf),len(im_idx_list_onlydf_val))        

5331 1335


In [0]:
def name_and_mask(start_idx):
    #in data set, each images has 4 rows, this grabs all 4 and makes sure image name matches
    
    col = start_idx
    img_names = [str(i).split("_")[0] for i in train_df.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError
    
    # This takes the 4 values of tagged pixels for each of the 4 defect tags 
    #for the current image
    labels = train_df.iloc[col:col+4, 1]
    
    #makes an empty mask that is 256x1600 pixels with 4 layers for each pixel
    mask = np.zeros((256, 1600, 4), dtype=np.uint8)

    
    #
    for idx, label in enumerate(labels.values):
        
# 4 times, once for each layer, the mask label is processed
# the output will leave all 0's for the mask we made above if there is no code
# or it will be converted to changing the mask on that layer
        
        if label is not np.nan:
            mask_label = np.zeros(1600*256, dtype=np.uint8)
            label = label.split(" ")

#makes a list out of non-zero labels, alternating between positions and lengths
            
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            
#makes lists of positions and lengths by iterating every other, 
#and forces them to become int          
            
            for pos, le in zip(positions, length):
                mask_label[pos-1:pos+le-1] = 1
            mask[:, :, idx] = mask_label.reshape(256, 1600, order='F')
# the positions called in label are turned to 1 in the mask for this layer
    
    return img_names[0], mask

In [0]:
class TFSCGen(tf.keras.utils.Sequence):
    """Generator class for Tensorflow Speech Competition data
    #Make a full table of 4 masks per image with dims [numpix,vertpix,horpix,masks(4)]
    args:
        - gen_path (patlib.Path): a path pointing to a directory containing training examples
            examples should be stored in format `gen_path/[class_name]/[file_name].wav`
        - batch_size (int): size of batches to return
        - shuffle (bool): whether or not data is to be shuffled between batches
    
    """
    
    
    def __init__(self, gen_path=im_idx_list_onlydf, batch_size = 24, shuffle=True):
        
        
        self.gen_files = im_idx_list_onlydf
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        if self.shuffle:
            random.shuffle(self.gen_files)

    def __len__(self):
        """returns the number of examples"""
        
        return int(np.ceil(len(self.gen_files) / float(self.batch_size)))
    
    def on_epoch_end(self):
        """shuffles data after an epoch runs (but only if self.shuffle is set)"""
        
        if self.shuffle:
            random.shuffle(self.gen_files)
            

    def __getitem__(self, idx):
        """function to return the batch given the batch index
        
        args:
            idx (int): this is the batch index generated by keras
        
        """
        
        
        start_idx = idx*self.batch_size
        batch_rows = self.gen_files[start_idx:start_idx+self.batch_size]
        
        x=[]
        for idx in batch_rows:    
            name, mask = name_and_mask(idx)
            abs_path = "/content/drive/My Drive/Steel/train_images/"
            #need to generalize this for tests
            
            filename=abs_path+name
            impath = Path(filename)
            img = cv2.imread(filename)
            x.append(img.astype(np.float32))
        x=np.stack(x, axis=0)   
        x=x/255
        
        
        y=[]
        for idx in batch_rows:
            pix,masks=name_and_mask(idx)
            indy=int(idx/4)
            y.append(masks.astype(np.float32))
        y=np.stack(y, axis=0)  
        
        
        
        return x,y

In [0]:
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tensorflow.keras.losses.binary_crossentropy(y_true, y_pred)  + Dice_Loss(y_true, y_pred)

bce = tf.keras.losses.BinaryCrossentropy()
  


In [0]:
n=8
g=2

In [0]:
def get_unet2():
    inputs = Input((256, 1600, 3))
    
    conv1 = Conv2D(n*g^0, (3, 3), strides=(2,2), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(n*g^0, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    
    conv2 = Conv2D(n*g^1, (3, 3), strides=(2,2), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(n*g^1, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(n*g^2, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(n*g^2, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(n*g^3, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(n*g^3, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(n*g^4, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(n*g^4, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv5)
    
    
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(n*g^3, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(n*g^3, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv6)
    
    
    #up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    #up7 =  UpSampling2D(size=(2, 2))(up7)
    conv7 = Conv2D(n*g^2, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(n*g^2, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv7)
    
    
    #up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    up8 =  UpSampling2D(size=(2, 2))(up8)
    conv8 = Conv2D(n*g^1, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(n*g^1, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization(axis=3, momentum=0.99, epsilon=0.001, center=True, scale=True)(conv8)
    
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    #up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    up9 =  UpSampling2D(size=(2, 2))(up9)
    conv9 = Conv2D(n*g^0, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(n*g^0, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(4, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(), loss=bce, metrics=[dice_coef])

    return model

In [20]:
unet_model = get_unet2()

unet_model.summary()
#check dimensions match expected output. 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 1600, 3 0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 128, 800, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 128, 800, 16) 2320        conv2d_39[0][0]                  
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 128, 800, 16) 64          conv2d_40[0][0]                  
____________________________________________________________________________________________

In [0]:
def train_and_predict():
  
    #Creating and compiling model
    
    
    model = get_unet2()
    model_checkpoint = ModelCheckpoint('/content/drive/My Drive/Steel/weights_norm.h5', monitor='val_loss', save_best_only=True)

    
    # Fitting model
    
    model.fit(TFSCGen(), batch_size=24, epochs=45, validation_data=TFSCGen(im_idx_list_onlydf_val), verbose=1, shuffle=True,
              
              callbacks=[model_checkpoint,
                   ReduceLROnPlateau(patience=3, verbose=True), 
                   EarlyStopping(patience=7, restore_best_weights=True, verbose=True)
               ])
    
    

In [22]:
train_and_predict()

Epoch 1/45
223/223 [==============================] - 1497s 7s/step - loss: 0.0723 - dice_coef: 0.1810 - val_loss: 0.1040 - val_dice_coef: 0.0080
Epoch 2/45
223/223 [==============================] - 523s 2s/step - loss: 0.0391 - dice_coef: 0.3236 - val_loss: 0.0589 - val_dice_coef: 0.2128
Epoch 3/45
223/223 [==============================] - 521s 2s/step - loss: 0.0361 - dice_coef: 0.3717 - val_loss: 0.0423 - val_dice_coef: 0.3332
Epoch 4/45
223/223 [==============================] - 526s 2s/step - loss: 0.0340 - dice_coef: 0.4002 - val_loss: 0.0315 - val_dice_coef: 0.4251
Epoch 5/45
223/223 [==============================] - 522s 2s/step - loss: 0.0319 - dice_coef: 0.4315 - val_loss: 0.0469 - val_dice_coef: 0.3291
Epoch 6/45
223/223 [==============================] - 525s 2s/step - loss: 0.0302 - dice_coef: 0.4564 - val_loss: 0.0292 - val_dice_coef: 0.4590
Epoch 7/45
223/223 [==============================] - 522s 2s/step - loss: 0.0305 - dice_coef: 0.4595 - val_loss: 0.0298 - val_di